In [1]:
# NYT Article Data Parser

import csv
from collections import defaultdict


# pull in the data from the Boydstun study
data_path = '/home/ryan/work/nytparser/data/'
data_file = 'Boydstun_random_10000.csv'
#data_file = 'Boydstun_NYT_FrontPage_Dataset TEST9Items.csv'
data_location = data_path + data_file

# topic codes used in the Boydstun study
boydstun_topic_codes = {'1':'Macroeconomics',
                        '2':'Civil Rights, Minority Issues, and Civil Liberties',
                        '3':'Health',
                        '4':'Agriculture',
                        '5':'Labor, Employment, and Immigration',
                        '6':'Education',
                        '7':'Environment',
                        '8':'Energy',
                        "9":'Quality of Life',
                        '10':'Transportation',
                        '12':'Law, Crime, and Family Issues',
                        '13':'Social Welfare',
                        '14':'Community Development and Housing Issues',
                        '15':'Banking, Finance, and Domestic Commerce',
                        '16':'Defense',
                        '17':'Space, Science, Technology and Communications',
                        '18':'Foreign Trade',
                        '19':'International Affairs and Foreign Aid',
                        '20':'Government Operations',
                        '21':'Public Lands and Water Management',
                        '24':'State and Local Government Administration',
                        '26':'Weather and Natural Disasters',
                        '27':'Fires',
                        '28':'Arts and Entertainment',
                        '29':'Sports and Recreation',
                        '30':'Death Notices',
                        '31':'Churches and Religion',
                        '99':'Other, Miscellaneous, and Human Interest',
                       }

boydstun_columns = defaultdict(list)

with open(data_location, newline='') as csvfile:
    rdr = csv.DictReader(csvfile)
    for row in rdr:
        for(k,v) in row.items():
            boydstun_columns[k].append(v)

boydstun_article_titles = boydstun_columns['title']
boydstun_article_classes = boydstun_columns['topic_2digit']

In [2]:
import urllib

def URL_title(title):
    
    #title = "\"" + title + "\""
    return urllib.parse.quote("\"" + title + "\"")
       
#URL_encoded_titles = []
#
#for title in boydstun_article_titles:
#    title = "\"" + title + "\""
#    #print(title)
#    URL_encoded_title = urllib.parse.quote(title)
#    #print(URL_encoded_title)
#    URL_encoded_titles.append(URL_encoded_title)

#print(URL_encoded_titles)

In [3]:
import json
import time
import pickle

# Get article data from NYT Developer API

# Build the query

# For reference
#base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?'
#API_key = 'MEt3Ki6vTVUvG4unee31Sb6MuSq1ACVO'
# Test URL
#url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=MEt3Ki6vTVUvG4unee31Sb6MuSq1ACVO&begin_date=19960101&end_date=20061231&fq=headline:(%22Tax%20Panel%20Says%20Popular%20Breaks%20Should%20Be%20Cut%22)&fq=print_page:1'



print("Found " + str(len(boydstun_article_titles)) + " article titles to search for.")

article_descriptions = []
output_list = []
calls = 0
successful_calls = 0

for title in boydstun_article_titles:
    
    # Build a list of article descriptions
    print("---> Fetching desriptions and class for article " + str(calls))
    #print(title)
            
    try:
        
        # Get an article
        url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=MEt3Ki6vTVUvG4unee31Sb6MuSq1ACVO&begin_date=19960101&end_date=20061231&fq=headline:(' + URL_title(title) + ')&fq=print_page:1'
        response = urllib.request.urlopen(url)
        json_response = json.loads(response.read())
        #print(json.dumps(json_dict, indent=4))
        
        # Extract available descriptions from each doc
        for doc in json_response['response']['docs']:
           
            if 'snippet' in doc:
                #print(doc['snippet'])
                snippet = doc['snippet']
            else:
                snippet = ''

            if 'lead_paragraph' in doc:
                #print(doc['lead_paragraph'])
                lead_paragraph = doc['lead_paragraph']
            else:
                lead_paragraph = ''

            article_descriptions.append(title + " " + snippet + " " + lead_paragraph)
            
            article_dict = {}
            article_dict['text'] = title + " " + snippet + " " + lead_paragraph
            article_dict['label'] = str(boydstun_topic_codes[boydstun_article_classes[calls]])
            output_list.append(article_dict)
            
            # preserve state
            pickle_file = open(data_location + '.pkl', 'wb')
            pickle.dump(output_list, pickle_file)
            pickle_file.close()
            
            with open(data_location + '.json', 'w') as outfile:
                json.dump(output_list, outfile, separators=(',', ':'))

            successful_calls += 1
            print("-----> Successful calls: " + str(successful_calls))

    except Exception as e:
        print(type(e))

    calls += 1
    print("-----> Calls: " + str(calls))
    
    time.sleep(6) #limit enforced by NYT

print("Articles pulled: " + str(len(output_list)))
#print(article_descriptions)

Found 10000 article titles to search for.
---> Fetching desriptions and class for article 0
-----> Successful calls: 1
-----> Calls: 1
---> Fetching desriptions and class for article 1
-----> Successful calls: 2
-----> Calls: 2
---> Fetching desriptions and class for article 2
-----> Successful calls: 3
-----> Calls: 3
---> Fetching desriptions and class for article 3
-----> Successful calls: 4
-----> Calls: 4
---> Fetching desriptions and class for article 4
-----> Successful calls: 5
-----> Calls: 5
---> Fetching desriptions and class for article 5
-----> Successful calls: 6
-----> Calls: 6
---> Fetching desriptions and class for article 6
-----> Successful calls: 7
-----> Calls: 7
---> Fetching desriptions and class for article 7
-----> Successful calls: 8
-----> Calls: 8
---> Fetching desriptions and class for article 8
-----> Successful calls: 9
-----> Calls: 9
---> Fetching desriptions and class for article 9
-----> Calls: 10
---> Fetching desriptions and class for article 10
---

---> Fetching desriptions and class for article 82
-----> Successful calls: 94
-----> Calls: 83
---> Fetching desriptions and class for article 83
-----> Calls: 84
---> Fetching desriptions and class for article 84
-----> Successful calls: 95
-----> Calls: 85
---> Fetching desriptions and class for article 85
-----> Calls: 86
---> Fetching desriptions and class for article 86
-----> Successful calls: 96
-----> Calls: 87
---> Fetching desriptions and class for article 87
-----> Calls: 88
---> Fetching desriptions and class for article 88
-----> Successful calls: 97
-----> Calls: 89
---> Fetching desriptions and class for article 89
-----> Successful calls: 98
-----> Calls: 90
---> Fetching desriptions and class for article 90
-----> Calls: 91
---> Fetching desriptions and class for article 91
-----> Successful calls: 99
-----> Calls: 92
---> Fetching desriptions and class for article 92
-----> Successful calls: 100
-----> Calls: 93
---> Fetching desriptions and class for article 93
----

-----> Calls: 171
---> Fetching desriptions and class for article 171
-----> Successful calls: 164
-----> Calls: 172
---> Fetching desriptions and class for article 172
-----> Successful calls: 165
-----> Calls: 173
---> Fetching desriptions and class for article 173
-----> Successful calls: 166
-----> Calls: 174
---> Fetching desriptions and class for article 174
-----> Successful calls: 167
-----> Calls: 175
---> Fetching desriptions and class for article 175
-----> Successful calls: 168
-----> Calls: 176
---> Fetching desriptions and class for article 176
-----> Successful calls: 169
-----> Calls: 177
---> Fetching desriptions and class for article 177
-----> Successful calls: 170
-----> Calls: 178
---> Fetching desriptions and class for article 178
-----> Successful calls: 171
-----> Calls: 179
---> Fetching desriptions and class for article 179
-----> Successful calls: 172
-----> Calls: 180
---> Fetching desriptions and class for article 180
-----> Successful calls: 173
-----> Cal

-----> Successful calls: 245
-----> Calls: 255
---> Fetching desriptions and class for article 255
-----> Successful calls: 246
-----> Calls: 256
---> Fetching desriptions and class for article 256
-----> Successful calls: 247
-----> Calls: 257
---> Fetching desriptions and class for article 257
-----> Calls: 258
---> Fetching desriptions and class for article 258
-----> Successful calls: 248
-----> Calls: 259
---> Fetching desriptions and class for article 259
-----> Successful calls: 249
-----> Calls: 260
---> Fetching desriptions and class for article 260
-----> Successful calls: 250
-----> Calls: 261
---> Fetching desriptions and class for article 261
-----> Successful calls: 251
-----> Calls: 262
---> Fetching desriptions and class for article 262
-----> Calls: 263
---> Fetching desriptions and class for article 263
-----> Calls: 264
---> Fetching desriptions and class for article 264
-----> Successful calls: 252
-----> Calls: 265
---> Fetching desriptions and class for article 26

---> Fetching desriptions and class for article 337
-----> Calls: 338
---> Fetching desriptions and class for article 338
-----> Successful calls: 329
-----> Calls: 339
---> Fetching desriptions and class for article 339
-----> Successful calls: 330
-----> Calls: 340
---> Fetching desriptions and class for article 340
-----> Successful calls: 331
-----> Calls: 341
---> Fetching desriptions and class for article 341
-----> Successful calls: 332
-----> Calls: 342
---> Fetching desriptions and class for article 342
-----> Successful calls: 333
-----> Calls: 343
---> Fetching desriptions and class for article 343
-----> Calls: 344
---> Fetching desriptions and class for article 344
-----> Successful calls: 334
-----> Calls: 345
---> Fetching desriptions and class for article 345
-----> Successful calls: 335
-----> Calls: 346
---> Fetching desriptions and class for article 346
-----> Successful calls: 336
-----> Calls: 347
---> Fetching desriptions and class for article 347
-----> Successfu

...

<class 'urllib.error.HTTPError'>
-----> Calls: 6172
---> Fetching desriptions and class for article 6172
<class 'urllib.error.HTTPError'>
-----> Calls: 6173
---> Fetching desriptions and class for article 6173
<class 'urllib.error.HTTPError'>
-----> Calls: 6174
---> Fetching desriptions and class for article 6174
<class 'urllib.error.HTTPError'>
-----> Calls: 6175
---> Fetching desriptions and class for article 6175
<class 'urllib.error.HTTPError'>
-----> Calls: 6176
---> Fetching desriptions and class for article 6176
<class 'urllib.error.HTTPError'>
-----> Calls: 6177
---> Fetching desriptions and class for article 6177
<class 'urllib.error.HTTPError'>
-----> Calls: 6178
---> Fetching desriptions and class for article 6178
<class 'urllib.error.HTTPError'>
-----> Calls: 6179
---> Fetching desriptions and class for article 6179
<class 'urllib.error.HTTPError'>
-----> Calls: 6180
---> Fetching desriptions and class for article 6180
<class 'urllib.error.HTTPError'>
-----> Calls: 6181
---

---> Fetching desriptions and class for article 6250
<class 'urllib.error.HTTPError'>
-----> Calls: 6251
---> Fetching desriptions and class for article 6251
<class 'urllib.error.HTTPError'>
-----> Calls: 6252
---> Fetching desriptions and class for article 6252
<class 'urllib.error.HTTPError'>
-----> Calls: 6253
---> Fetching desriptions and class for article 6253
<class 'urllib.error.HTTPError'>
-----> Calls: 6254
---> Fetching desriptions and class for article 6254
<class 'urllib.error.HTTPError'>
-----> Calls: 6255
---> Fetching desriptions and class for article 6255
<class 'urllib.error.HTTPError'>
-----> Calls: 6256
---> Fetching desriptions and class for article 6256
<class 'urllib.error.HTTPError'>
-----> Calls: 6257
---> Fetching desriptions and class for article 6257
<class 'urllib.error.HTTPError'>
-----> Calls: 6258
---> Fetching desriptions and class for article 6258
<class 'urllib.error.HTTPError'>
-----> Calls: 6259
---> Fetching desriptions and class for article 6259
<c

<class 'urllib.error.HTTPError'>
-----> Calls: 6329
---> Fetching desriptions and class for article 6329
<class 'urllib.error.HTTPError'>
-----> Calls: 6330
---> Fetching desriptions and class for article 6330
<class 'urllib.error.HTTPError'>
-----> Calls: 6331
---> Fetching desriptions and class for article 6331
<class 'urllib.error.HTTPError'>
-----> Calls: 6332
---> Fetching desriptions and class for article 6332
<class 'urllib.error.HTTPError'>
-----> Calls: 6333
---> Fetching desriptions and class for article 6333
<class 'urllib.error.HTTPError'>
-----> Calls: 6334
---> Fetching desriptions and class for article 6334
<class 'urllib.error.HTTPError'>
-----> Calls: 6335
---> Fetching desriptions and class for article 6335
<class 'urllib.error.HTTPError'>
-----> Calls: 6336
---> Fetching desriptions and class for article 6336
<class 'urllib.error.HTTPError'>
-----> Calls: 6337
---> Fetching desriptions and class for article 6337
<class 'urllib.error.HTTPError'>
-----> Calls: 6338
---

---> Fetching desriptions and class for article 6407
<class 'urllib.error.HTTPError'>
-----> Calls: 6408
---> Fetching desriptions and class for article 6408
<class 'urllib.error.HTTPError'>
-----> Calls: 6409
---> Fetching desriptions and class for article 6409
<class 'urllib.error.HTTPError'>
-----> Calls: 6410
---> Fetching desriptions and class for article 6410
<class 'urllib.error.HTTPError'>
-----> Calls: 6411
---> Fetching desriptions and class for article 6411
<class 'urllib.error.HTTPError'>
-----> Calls: 6412
---> Fetching desriptions and class for article 6412
<class 'urllib.error.HTTPError'>
-----> Calls: 6413
---> Fetching desriptions and class for article 6413
<class 'urllib.error.HTTPError'>
-----> Calls: 6414
---> Fetching desriptions and class for article 6414
<class 'urllib.error.HTTPError'>
-----> Calls: 6415
---> Fetching desriptions and class for article 6415
<class 'urllib.error.HTTPError'>
-----> Calls: 6416
---> Fetching desriptions and class for article 6416
<c

KeyboardInterrupt: 

In [ ]:
# Transform to useful format

"""
Goal output:

{"text": "some text", "label": "politics"}
{"text": "some other text", "label": "science"}
"""

output_list = []

print('Found ' + str(len(article_descriptions)) + ' articles.')


for i in range(len(article_descriptions)):
    
    print('---> Working on article ' + str(i) + ': topic: ' + str(boydstun_topic_codes[boydstun_article_classes[i]]))
    
    article_dict = {}
    
    try:
        article_dict['text'] = article_descriptions[i]
        article_dict['label'] = str(boydstun_topic_codes[boydstun_article_classes[i]])
        output_list.append(article_dict)

    except Exception as e:
        print(type(e))
        
with open(data_location + '.json', 'w') as outfile:
    json.dump(output_list, outfile, separators=(',', ':'))